In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

### Network Architecture

In [5]:
import torch.nn as nn
from collections import OrderedDict
from torchsummary import summary

In [6]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


### Training Function

In [7]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss_func = nn.NLLLoss()
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx%args.log_interval == 0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset), 100.*batch_idx/len(train_loader), loss.item()))

### Testing Function

In [8]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = nn.NLLLoss()
            test_loss += F.nll_loss(output, target, size_average=False).item() # Sum up the batch loss
            pred = output.max(1, keepdim=True)[1] # Get the index of the max log probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100.*correct/len(test_loader.dataset)))

### Main Function

In [9]:
def main():
    # Training settings
    
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=20, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    args = parser.parse_args([])
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Network()
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [10]:
if __name__ == '__main__':
    main()

100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:07<00:00, 1254123.23it/s]


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw



100%|████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<?, ?it/s]


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw



100%|████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:02<00:00, 778484.43it/s]


...

Train Epoch: 14 [4480/60000 (7%)]	Loss: 0.119941
Train Epoch: 14 [5120/60000 (9%)]	Loss: 0.069351
Train Epoch: 14 [5760/60000 (10%)]	Loss: 0.083064
Train Epoch: 14 [6400/60000 (11%)]	Loss: 0.017033
Train Epoch: 14 [7040/60000 (12%)]	Loss: 0.080095
Train Epoch: 14 [7680/60000 (13%)]	Loss: 0.118600
Train Epoch: 14 [8320/60000 (14%)]	Loss: 0.166159
Train Epoch: 14 [8960/60000 (15%)]	Loss: 0.269343
Train Epoch: 14 [9600/60000 (16%)]	Loss: 0.218993
Train Epoch: 14 [10240/60000 (17%)]	Loss: 0.195344
Train Epoch: 14 [10880/60000 (18%)]	Loss: 0.131679
Train Epoch: 14 [11520/60000 (19%)]	Loss: 0.181582
Train Epoch: 14 [12160/60000 (20%)]	Loss: 0.148459
Train Epoch: 14 [12800/60000 (21%)]	Loss: 0.099498
Train Epoch: 14 [13440/60000 (22%)]	Loss: 0.045427
Train Epoch: 14 [14080/60000 (23%)]	Loss: 0.193063
Train Epoch: 14 [14720/60000 (25%)]	Loss: 0.055053
Train Epoch: 14 [15360/60000 (26%)]	Loss: 0.119026
Train Epoch: 14 [16000/60000 (27%)]	Loss: 0.083212
Train Epoch: 14 [16640/60000 (28%)]	Loss: 

Train Epoch: 15 [47360/60000 (79%)]	Loss: 0.031430
Train Epoch: 15 [48000/60000 (80%)]	Loss: 0.223273
Train Epoch: 15 [48640/60000 (81%)]	Loss: 0.085892
Train Epoch: 15 [49280/60000 (82%)]	Loss: 0.098492
Train Epoch: 15 [49920/60000 (83%)]	Loss: 0.089266
Train Epoch: 15 [50560/60000 (84%)]	Loss: 0.312541
Train Epoch: 15 [51200/60000 (85%)]	Loss: 0.091939
Train Epoch: 15 [51840/60000 (86%)]	Loss: 0.069854
Train Epoch: 15 [52480/60000 (87%)]	Loss: 0.196612
Train Epoch: 15 [53120/60000 (88%)]	Loss: 0.183830
Train Epoch: 15 [53760/60000 (90%)]	Loss: 0.196854
Train Epoch: 15 [54400/60000 (91%)]	Loss: 0.157783
Train Epoch: 15 [55040/60000 (92%)]	Loss: 0.159756
Train Epoch: 15 [55680/60000 (93%)]	Loss: 0.139511
Train Epoch: 15 [56320/60000 (94%)]	Loss: 0.080887
Train Epoch: 15 [56960/60000 (95%)]	Loss: 0.119049
Train Epoch: 15 [57600/60000 (96%)]	Loss: 0.219397
Train Epoch: 15 [58240/60000 (97%)]	Loss: 0.213662
Train Epoch: 15 [58880/60000 (98%)]	Loss: 0.088520
Train Epoch: 15 [59520/60000 (9

Train Epoch: 17 [29440/60000 (49%)]	Loss: 0.173446
Train Epoch: 17 [30080/60000 (50%)]	Loss: 0.059750
Train Epoch: 17 [30720/60000 (51%)]	Loss: 0.223482
Train Epoch: 17 [31360/60000 (52%)]	Loss: 0.155742
Train Epoch: 17 [32000/60000 (53%)]	Loss: 0.068730
Train Epoch: 17 [32640/60000 (54%)]	Loss: 0.080090
Train Epoch: 17 [33280/60000 (55%)]	Loss: 0.075120
Train Epoch: 17 [33920/60000 (57%)]	Loss: 0.114853
Train Epoch: 17 [34560/60000 (58%)]	Loss: 0.055297
Train Epoch: 17 [35200/60000 (59%)]	Loss: 0.263148
Train Epoch: 17 [35840/60000 (60%)]	Loss: 0.120572
Train Epoch: 17 [36480/60000 (61%)]	Loss: 0.215371
Train Epoch: 17 [37120/60000 (62%)]	Loss: 0.050424
Train Epoch: 17 [37760/60000 (63%)]	Loss: 0.130144
Train Epoch: 17 [38400/60000 (64%)]	Loss: 0.214777
Train Epoch: 17 [39040/60000 (65%)]	Loss: 0.027279
Train Epoch: 17 [39680/60000 (66%)]	Loss: 0.173391
Train Epoch: 17 [40320/60000 (67%)]	Loss: 0.060295
Train Epoch: 17 [40960/60000 (68%)]	Loss: 0.218800
Train Epoch: 17 [41600/60000 (6

Train Epoch: 19 [11520/60000 (19%)]	Loss: 0.344712
Train Epoch: 19 [12160/60000 (20%)]	Loss: 0.178165
Train Epoch: 19 [12800/60000 (21%)]	Loss: 0.114427
Train Epoch: 19 [13440/60000 (22%)]	Loss: 0.119189
Train Epoch: 19 [14080/60000 (23%)]	Loss: 0.105415
Train Epoch: 19 [14720/60000 (25%)]	Loss: 0.023849
Train Epoch: 19 [15360/60000 (26%)]	Loss: 0.034787
Train Epoch: 19 [16000/60000 (27%)]	Loss: 0.156630
Train Epoch: 19 [16640/60000 (28%)]	Loss: 0.011121
Train Epoch: 19 [17280/60000 (29%)]	Loss: 0.073314
Train Epoch: 19 [17920/60000 (30%)]	Loss: 0.063658
Train Epoch: 19 [18560/60000 (31%)]	Loss: 0.110942
Train Epoch: 19 [19200/60000 (32%)]	Loss: 0.161214
Train Epoch: 19 [19840/60000 (33%)]	Loss: 0.113958
Train Epoch: 19 [20480/60000 (34%)]	Loss: 0.293067
Train Epoch: 19 [21120/60000 (35%)]	Loss: 0.115620
Train Epoch: 19 [21760/60000 (36%)]	Loss: 0.123398
Train Epoch: 19 [22400/60000 (37%)]	Loss: 0.180935
Train Epoch: 19 [23040/60000 (38%)]	Loss: 0.173695
Train Epoch: 19 [23680/60000 (3

Train Epoch: 20 [54400/60000 (91%)]	Loss: 0.239151
Train Epoch: 20 [55040/60000 (92%)]	Loss: 0.092728
Train Epoch: 20 [55680/60000 (93%)]	Loss: 0.036687
Train Epoch: 20 [56320/60000 (94%)]	Loss: 0.123641
Train Epoch: 20 [56960/60000 (95%)]	Loss: 0.065756
Train Epoch: 20 [57600/60000 (96%)]	Loss: 0.037675
Train Epoch: 20 [58240/60000 (97%)]	Loss: 0.109793
Train Epoch: 20 [58880/60000 (98%)]	Loss: 0.183724
Train Epoch: 20 [59520/60000 (99%)]	Loss: 0.186517

Test set: Average loss: 0.0374, Accuracy: 9877/10000 (99%)



In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("")